In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

def fc_to_image(fc, property, scale = 1000):
    # Convert the feature collection to an image
    return fc.reduceToImage(
            properties = [property],
            reducer = ee.Reducer.first()
        ).reproject(
            crs = 'EPSG:4326',  # or match your source CRS
            scale = scale
        ).rename(property)


def export_image(image, description):
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = description,
        fileNamePrefix = description,
        region = roi,
        scale = image.projection().nominalScale(),
        maxPixels = 1e13,
        crs = 'EPSG:4326',
        fileFormat = 'GeoTIFF'
    )
    task.start()

## Export asymptotes for Figure 2

In [ ]:
nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()

quarters_ecoreg_biomass = ee.Image(f"{data_folder}/quarters_ecoreg_biomass").select("quarter_biomass").selfMask()

biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask").selfMask()

# nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()


# export_image(nearest_mature, "nearest_mature")
# export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass")
# export_image(biomes_mask, "biome_mask")

## Export predictions for Figure 4

In [ ]:

# Predictions for 2050
pred_lag_2050 = ee.FeatureCollection(f"{data_folder}/results/pred_lag_2050")
pred_lag_2050 = fc_to_image(pred_lag_2050, "pred_lag_2", scale=1000)

pred_delta = pred_lag_2050.subtract(biomass).rename("pred_delta")

# Aggregate the high-resolution pixels into the 10 km grid
pred_delta_10k = pred_delta.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 120000,
    bestEffort = True # Use all pixels that can fit in the larger pixel
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("pred_delta")

vis_params = {'min': -50, 'max': 120, 'palette': ['red', 'green']}
map = geemap.Map(basemap = 'CartoDB.Positron')
# map.centerObject(roi, 6)
# map.addLayer(biomass, vis_params, "biomass_2020")
# map.addLayer(pred_lag_2050, vis_params, "biomass_2050")
# map.addLayer(asymptote, vis_params, "asymptote")
map.addLayer(pred_delta, vis_params, "pred_delta")
map.addLayer(pred_delta_10k, vis_params, "pred_delta_10k")
map.add_colorbar(vis_params, label = "biomass (Mg/ha)", layer_name = "biomass")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export Extended Data Maps

In [ ]:
# Absolute error
relative_growth = ee.FeatureCollection(f"{data_folder}/results/relative_growth")
relative_growth = fc_to_image(relative_growth, "percentage")

vis_params = {'min': 40, 'max': 60, 'palette': ['yellow', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(relative_growth, vis_params, "relative_growth")
map.add_colorbar(vis_params, label = "Percentage of asymptote", layer_name = "relative_growth")
map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Absolute error
absolute_error = ee.FeatureCollection(f"{data_folder}/results/absolute_error")
absolute_error = fc_to_image(absolute_error, "error")

# Percent error
percent_error = ee.FeatureCollection(f"{data_folder}/results/percent_error")
percent_error = fc_to_image(percent_error, "error")

vis_params = {'min': -100, 'max': 100, 'palette': ['red', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(absolute_error, vis_params, "absolute_error")
map.addLayer(percent_error, vis_params, "percent_error")
map.add_colorbar(vis_params, label = "Absolute error", layer_name = "absolute_error")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…